In [ ]:
__author__ = 'shangxing'
#!/usr/bin/env python
#
#  Python program that will create the sample solution
#
#  @author: lak@climate.com
#  @version: $Id:$
#
#  you may have to install the master branch of 'dask' using:
#     pip install --upgrade git+https://github.com/blaze/dask
#
	 
# Rate from KDP and ZDR

# Equation form from Ryzhkov and Zrnic, 1995, with coefficents determined by Brandes (personal communication w/ M. Dixon) for the tropical situation of CP2 radar and disdrometer data near Brisbane.

#       RATE_KDP_ZDR =
#         sign(KDP) * kdpzdr_aa * (|KDP| ** kdpzdr_bb) *
#        (ZDR  ** kdpzdr_cc)

#   where

#     kdpzdr_aa = 136
#     kdpzdr_bb = 0.968
#     kdpzdr_cc = -2.86


import time
import pandas as pd
import numpy as np
import sys
import pickle as df
import math

In [ ]:

# change the location of the downloaded test file as necessary.
infile="../data/test.csv"
#infile="kaggle/sample.csv"
outfile="../data/new_solution_11_12.csv"

In [1]:
# read file
t = time.time()
alldata = pd.read_csv(infile)



NameError: name 'time' is not defined

In [ ]:
kdpzdr_aa = 136
kdpzdr_bb = 0.968
kdpzdr_cc = -2.86

#       RATE_KDP_ZDR =
#         sign(KDP) * kdpzdr_aa * (|KDP| ** kdpzdr_bb) *
#        (ZDR  ** kdpzdr_cc)

def rainRate_KDP_ZDR(kdp,zdr, minutes_past):
    #print "Estimating rainfall from {0} observations".format(len(minutes_past))
    # how long is each observation valid?
    valid_time = np.zeros_like(minutes_past)
    valid_time[0] = minutes_past.iloc[0]
    for n in range(1, len(minutes_past)):
        valid_time[n] = minutes_past.iloc[n] - minutes_past.iloc[n-1]
    valid_time[-1] = valid_time[-1] + 60 - np.sum(valid_time)
    valid_time = valid_time / 60.0

    # sum up rainrate * validtime
    sum = 0
    for kdp_temp,zdr_temp, hours in zip(ref,zdr, valid_time):
        # See: https://en.wikipedia.org/wiki/DBZ_(meteorology)
        if np.isfinite(kdp_temp) and np.isfinite(zdr_temp):
           
            mmperhr = np.sign(kdp_temp)*kdpzdr_aa*(pow(np.abs(kdp_temp),kdpzdr_bb)*pow(zdr_temp,kdpzdr_cc)
            sum = sum + mmperhr * hours
    return sum


# each unique Id is an hour of data at some gauge
def myfunc(hour):

    hour = hour.sort('minutes_past', ascending=True)
    est = rainRate_KDP_ZDR(hour['Kdp'], hour['Zdr'],hour['minutes_past'])
    return est


#return(sum((sign(Kdp)*(40.6)*(abs(Kdp) **.866)*valid_time), na.rm=TRUE))
print 'Time elapsed: {:.4f} secs'.format(time.time()-t) 

In [ ]:
t = time.time()
estimates = alldata.groupby(alldata.Id).apply(lambda d: myfunc(d))

df = pd.DataFrame(estimates,columns=['Expected'])

df.to_csv(outfile, header=True)
print 'Time elapsed: {:.4f} secs'.format(time.time()-t) 